# Bioinformatika Lab. 1

In [28]:
from Bio import SeqIO
for seq_record in SeqIO.parse("plazmide.fasta", "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

plazmide
Seq('TTGCGCCGGCGAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATC...CCT', SingleLetterAlphabet())
5329


In [29]:
rec = SeqIO.read("plazmide.fasta", "fasta")

In [32]:
print(rec)

ID: plazmide
Name: plazmide
Description: plazmide
Number of features: 0
Seq('TTGCGCCGGCGAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATC...CCT', SingleLetterAlphabet())


In [25]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

protein_seq = Seq(str(rec.seq), IUPAC.protein)

In [24]:
IUPAC.protein

IUPACProtein()